# Fetch Open Datasets of PV locations

Many of these datasets are located in [Zenodo](https://zenodo.org/), a general-purpose open-access repository developed under the European OpenAIRE program and operated by CERN. Others are hosted in figshare, a web-based platform for sharing research data and other types of content. The rest are hosted in GitHub repositories or other open-access platforms.The datasets are available in various formats, including CSV, GeoJSON, and shapefiles, and raster masks. We'll be using open-source Python libraries to download and process them into properly georeferenced geoparquet files
that will serve as a base for our duckdb tables that we'll manage with dbt

Here we list the dataset titles alongside their first author, DOI links, and their number of labels:
- "A solar panel dataset of very high resolution satellite imagery to support the Sustainable Development Goals" - C. Clark et al, 2023 | [paper DOI](https://doi.org/10.1038/s41597-023-02539-8) | [dataset DOI](https://doi.org/10.6084/m9.figshare.22081091.v3) | 2,542 object labels (per spatial resolution)
- "A global inventory of photovoltaic solar energy generating units" - L. Kruitwagen et al, 2021 | [paper DOI](https://doi.org/10.1038/s41586-021-03957-7) | [dataset DOI](https://doi.org/10.5281/zenodo.5005867) | 50,426 for training, cross-validation, and testing; 68,661 predicted polygon labels 
- "A harmonised, high-coverage, open dataset of solar photovoltaic installations in the UK" - D. Stowell et al, 2020 | [paper DOI](https://doi.org/10.1038/s41597-020-00739-0) | [dataset DOI](https://zenodo.org/records/4059881) | 265,418 data points (over 255,000 are stand-alone installations, 1067 solar farms, and rest are subcomponents within solar farms)
- "A crowdsourced dataset of aerial images with annotated solar photovoltaic arrays and installation metadata" - G. Kasmi, 2023 | [paper DOI](https://doi.org/10.1038/s41597-023-01951-4) | [dataset DOI](https://doi.org/10.5281/zenodo.6865878) | > 28K points of PV installations; 13K+ segmentation masks for PV arrays; metadata for 8K+ installations
- "Georectified polygon database of ground-mounted large-scale solar photovoltaic sites in the United States" - K. Sydny, 2023 | [paper DOI](https://doi.org/10.1038/s41597-023-02644-8) | [dataset DOI](https://www.sciencebase.gov/catalog/item/6671c479d34e84915adb7536) | 4186 data points (Note: these correspond to PV _facilities_ rather than individual panel arrays or objects and need filtering of duplicates with other datasets and further processing to extract the PV arrays in the facility)
- "Vectorized solar photovoltaic installation dataset across China in 2015 and 2020" - J. Liu et al, 2024 | [paper DOI](https://doi.org/10.1038/s41597-024-04356-z) | [dataset link](https://github.com/qingfengxitu/ChinaPV) | 3,356 PV labels (inspect quality!)
- "Multi-resolution dataset for photovoltaic panel segmentation from satellite and aerial imagery" - H. Jiang, 2021 | [paper DOI](https://doi.org/10.5194/essd-13-5389-2021) | [dataset DOI](https://doi.org/10.5281/zenodo.5171712) | 3,716 samples of PV data points
- "An Artificial Intelligence Dataset for Solar Energy Locations in India" - A. Ortiz, 2022 | [paper DOI](https://doi.org/10.1038/s41597-022-01499-9) | [dataset link 1](https://researchlabwuopendata.blob.core.windows.net/solar-farms/solar_farms_india_2021.geojson) or [dataset link 2](https://raw.githubusercontent.com/microsoft/solar-farms-mapping/refs/heads/main/data/solar_farms_india_2021_merged_simplified.geojson) | 117 geo-referenced points of solar installations across India
- "GloSoFarID: Global multispectral dataset for Solar Farm IDentification in satellite imagery" - Z. Yang, 2024 | [paper DOI](https://doi.org/10.48550/arXiv.2404.05180) | [dataset DOI](https://doi.org/10.5281/zenodo.10939099) | 6,793 PV samples across 3 years (double counting of samples)
- "Distributed solar photovoltaic array location and extent dataset for remote sensing object identification" - K. Bradbury, 2016 | [paper DOI](https://doi.org/10.1038/sdata.2016.106) | [dataset DOI](https://doi.org/10.6084/m9.figshare.3385780.v4) | polygon annotations for 19,433 PV modules in 4 cities in California, USA

In [1]:
import ipywidgets as widgets
from IPython.display import display
from dotenv import load_dotenv
from tqdm import tqdm

import numpy as np
import xarray as xr
# import matplotlib.pyplot as plt

import geopandas as gpd
import rasterio
import shapely
import pygeohash
import folium
import lonboard
# import openeo 
# import easystac
# import pydeck
# import easystac
# import cubo

import duckdb as dd 
import datahugger
import sciencebasepy

import os

In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout
import json

# create dict of metadata for datasets
# this will be used for interactive widget and managing downloads

# for maxar dataset
# Catalogue ID 1040050029DC8C00; use to find geospatial extent coords
# The geocoordinates for each solar panel object may be determined using the native resolution labels (found in the labels_native directory). 
# The center and width values for each object, along with the relative location information provided by the naming convention for each label, 
# may be used to determine the pixel coordinates for each object in the full, corresponding native resolution tile. 
# The pixel coordinates may be translated to geocoordinates using the EPSG:32633 coordinate system and the following geotransform for each tile:

# Tile 1: (307670.04, 0.31, 0.0, 5434427.100000001, 0.0, -0.31)
# Tile 2: (312749.07999999996, 0.31, 0.0, 5403952.860000001, 0.0, -0.31)
# Tile 3: (312749.07999999996, 0.31, 0.0, 5363320.540000001, 0.0, -0.31)
# see here on gdal format geotransform: https://gdal.org/en/stable/tutorials/geotransforms_tut.html

# look into adding dataset crs or projection to metadata dict
# note that most of these details are hardcoded and difficult to parse ahead of time
# load environment variables
load_dotenv()
dataset_metadata = {
    'deu_maxar_vhr_2023': {
        'doi': '10.6084/m9.figshare.22081091.v3',
        'repo': 'figshare',
        'compression': 'zip',
        'label_fmt': 'yolo_fmt_txt',
        'has_imgs': False,
        'label_count': 2542 # solar panel objects (ie not individual panels)
    },
    'uk_crowdsourced_pv_2020': {
        'doi': '10.5281/zenodo.4059881',
        'repo': 'zenodo',
        'compression': None,
        'label_fmt': 'geojson',
        'has_imgs': False,
        'label_count': 265418
    },
    # note for report later: Maxar Technologies (MT) was primarily used to determine the extent of solar arrays
    'usa_eia_large_scale_pv_2023': {
        'doi': '10.5281/zenodo.8038684',
        'repo': 'sciencebase',
        'compression': 'zip',
        'label_fmt': 'shapefile',
        'has_imgs': False,
        'label_count': 4186
    },
    'chn_med_res_pv_2024': {
        # using github files since zenodo shapefiles fail to load in QGIS
        'doi': 'https://github.com/qingfengxitu/ChinaPV/tree/main',
        'repo': 'github',
        'compression': None,
        'label_fmt': 'shapefile',
        'has_imgs': False,
        'label_count': 3356
    },
    'usa_cali_usgs_pv_2016': {
        'doi': '10.6084/m9.figshare.3385780.v4',
        'repo': 'figshare',
        'compression': None,
        'label_fmt': 'geojson',
        'has_imgs': False,
        'label_count': 19433
    },
    'chn_jiangsu_vhr_pv_2021': {
        'doi': '10.5281/zenodo.5171712',
        'repo': 'zenodo',
        'compression': 'zip',
        # look into geotransform details for processing these labels
        'label_fmt': 'pixel_mask',
        'has_imgs': True,
        'label_count': 3716
    },
    'ind_pv_solar_farms_2022': {
        'doi': 'https://raw.githubusercontent.com/microsoft/solar-farms-mapping/refs/heads/main/data/solar_farms_india_2021_merged_simplified.geojson',
        'repo': 'github',
        'compression': None,
        'label_fmt': 'geojson',
        'has_imgs': False,
        'label_count': 117
    },
    'fra_west_eur_pv_installations_2023': {
        'doi': '10.5281/zenodo.6865878',
        'repo': 'zenodo',
        'compression': 'zip',
        'label_fmt': 'geojson',
        'has_imgs': True, 
        'lambda_count': (13303, 7686)
    },
    'global_pv_inventory_sent2_spot_2021': {
        'doi': '10.5281/zenodo.5005867',
        'repo': 'zenodo',
        'compression': None,
        'label_fmt': 'geojson',
        'has_imgs': False,
        'label_count': 50426
    },
    'global_pv_inventory_sent2_2024': {
        'doi': '10.5281/zenodo.10939099',
        'repo': 'zenodo',
        'compression': None,
        'label_fmt': 'tf_record',
        'has_imgs': True, 
        'label_count': 6793
    }

}

dataset_choices = [
    # 'global_pv_inventory_sent2_2024',
    'global_pv_inventory_sent2_spot_2021',
    'fra_west_eur_pv_installations_2023',
    # 'ind_pv_solar_farms_2022',
    'usa_cali_usgs_pv_2016',
    # 'chn_med_res_pv_2024',
    'usa_eia_large_scale_pv_2023',
    'uk_crowdsourced_pv_2020',
    'deu_maxar_vhr_2023'   
]

# Initialize a list to store selected datasets
selected_datasets = dataset_choices.copy()

def format_dataset_info(dataset):
    """Create a formatted HTML table for dataset metadata"""
    metadata = dataset_metadata[dataset]
    
    # Create table with metadata
    html = f"""
    <style>
    .dataset-table {{
        border-collapse: collapse;
        width: 100%;
        font-family: Arial, sans-serif;
    }}
    .dataset-table th, .dataset-table td {{
        border: 1px solid #ddd;
        padding: 8px;
        text-align: left;
    }}
    .dataset-table th {{
        background-color: #f2f2f2;
        font-weight: bold;
    }}
    </style>
    <table class="dataset-table">
        <tr><th>Metadata</th><th>Value</th></tr>
        <tr><td>DOI/URL</td><td>{metadata['doi']}</td></tr>
        <tr><td>Repository</td><td>{metadata['repo']}</td></tr>
        <tr><td>Compression</td><td>{metadata['compression'] or 'None'}</td></tr>
        <tr><td>Label Format</td><td>{metadata['label_fmt']}</td></tr>
        <tr><td>Has Images</td><td>{metadata['has_imgs']}</td></tr>
        <tr><td>Label Count</td><td>{metadata.get('label_count', 'Unknown')}</td></tr>
    </table>
    """
    return html

# Create an accordion to display selected datasets
dataset_accordion = widgets.Accordion(children=[widgets.HTML(format_dataset_info(ds)) for ds in selected_datasets])
for i, ds in enumerate(selected_datasets):
    dataset_accordion.set_title(i, ds)

# Define a function to add or remove datasets
def manage_datasets(action, dataset=None):
    global selected_datasets, dataset_accordion
    
    if action == 'add' and dataset and dataset not in selected_datasets:
        selected_datasets.append(dataset)
    elif action == 'remove' and dataset and dataset in selected_datasets:
        selected_datasets.remove(dataset)
    
    # Update the accordion with current selections
    dataset_accordion.children = [widgets.HTML(format_dataset_info(ds)) for ds in selected_datasets]
    for i, ds in enumerate(selected_datasets):
        dataset_accordion.set_title(i, ds)
    
    f"Currently selected datasets: {len(selected_datasets)}")

# Create dropdown for available datasets
dataset_dropdown = widgets.Dropdown(
    options=list(dataset_metadata.keys()),
    description='Dataset:',
    disabled=False,
    layout=Layout(width='50%')
)

# Create buttons for actions
add_button = widgets.Button(description="Add Dataset", button_style='success')
remove_button = widgets.Button(description="Remove Dataset", button_style='danger')

# Define button click handlers
def on_add_clicked(b):
    manage_datasets('add', dataset_dropdown.value)

def on_remove_clicked(b):
    manage_datasets('remove', dataset_dropdown.value)

# Link buttons to handlers
add_button.on_click(on_add_clicked)
remove_button.on_click(on_remove_clicked)

## Dataset Selection Interface
#### Use the dropdown and buttons below to customize which solar panel datasets will be fetched and processed.
- Select a dataset from the dropdown
- Click "Add Dataset" to include it in processing
- Click "Remove Dataset" to exclude it
- View detailed metadata in the selected dataset's dropdown table

In [12]:
# Display the widgets
display(widgets.HBox([dataset_dropdown, add_button, remove_button]))
display(dataset_accordion)

Accordion(children=(HTML(value='\n    <style>\n    .dataset-table {\n        border-collapse: collapse;\n     …

In [13]:
# use the metadata to fetch the dataset files using datahugger
def fetch_dataset_files(dataset_name):
    metadata = dataset_metadata[dataset_name]
    doi = metadata['doi']
    repo = metadata['repo']
    compression = metadata['compression']
    label_fmt = metadata['label_fmt']

    # use datahugger to fetch files from most repos
    if repo in ['figshare', 'zenodo', 'github']:

        datahugger